In [1]:
import torch
from torch import nn,optim
from torchvision.models import resnet18
from data_utils.data import FaceMaskDetectorData
from torch.utils.data import DataLoader
from models.face_mask_detector import FaceMaskDetector
from settings import *
from torch.nn import functional as F

BATCH_SIZE = 128 
ETA = 1e-3
EPOCHS = 1


'\nclass FaceMaskDetector(nn.Module):\n\n    def __init__(self):\n        super(FaceMaskDetector, self).__init__()\n\n        resnet = resnet18(pretrained=True)\n        resnet = list(resnet.children())[:8]\n\n        self.features = nn.Sequential(*resnet)\n\n        self.detector = nn.Sequential(\n            nn.Conv2d(512, 1, (2, 2), stride=1, padding=0),\n            nn.Sigmoid()\n        )\n\n    def forward(self, x):\n        features = self.features(x)\n        preds = self.detector(features)\n        return preds\n'

In [2]:
def compute_accuracy(ps,labels):
    equality= (torch.round(ps)==labels).float()
    acc= equality.mean()
    return acc

def train_step(model,optimizer,x,y):
    
    optimizer.zero_grad()
    
    ps=model(x).squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc=compute_accuracy(ps,y)
    
    return loss.item(), acc.item()

def eval_step(model, x, y):
    
    ps = model(x).squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

import math
import time
import random

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [3]:
def train():
    trainset = FaceMaskDetectorData(mode="train", distortion=True, flip=True)
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = FaceMaskDetectorData(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    '''
    if CUDA is True:
        model = FaceMaskDetector().cuda()
    else:
        model = FaceMaskDetector()
    '''
    model = FaceMaskDetector()
    optimizer = optim.Adam(model.parameters())
    
    for e in range(EPOCHS):

        start_time = time.time() # 시작 시간 기록
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            y = y.float()
            '''
            if CUDA is True:
                x = x.cuda()
                y = y.cuda()
            '''
            loss, acc = train_step(model,optimizer, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            y = y.float()
            '''
            if CUDA is True:
                x = x.cuda()
                y = y.cuda()
            '''
            with torch.no_grad():
                loss, acc = eval_step(model, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        end_time = time.time() # 종료 시간 기록
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f'Epochs {e+1}/{EPOCHS} | Time: {epoch_mins}m {epoch_secs}s')
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")
        
        torch.save(model.state_dict(), FACE_MASK_DETECTOR_CKPT)

In [4]:
train()

C:\Users\Songwooseok\miniconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\Songwooseok\miniconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epochs 1/1 | Time: 4m 10s
Train loss: 0.06706640, train acc: 0.9777
Test loss: 0.01050043, test acc: 0.9971


In [1]:
from run import *

In [2]:
run()

C:\Users\Songwooseok\miniconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\Songwooseok\miniconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Songwooseok\BDML_STUDY2022\MaskFaceDetector\run.py:26: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(torch.from_numpy(img).float(),volatile=True)
C:\Users\Songwooseok\BDML_STUDY2022\MaskFaceDetec

NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
:)
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK!
NO MASK

KeyboardInterrupt: 